In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,Dropout,Flatten

In [7]:
train_data = []
labels = []
path = "./dataset/Train"
classes=43
for i in range(classes):
    path1=path+"/"+str(i)
    files = os.listdir(path1)
    for file in files:
        try:
            image = Image.open(path1+"/"+file)
            image  = image.resize((30,30))
            image = np.array(image)
            image = image/255
            train_data.append(image)
            labels.append(i)
        except:
            print("Error loading image")
train_data = np.array(train_data)
labels = np.array(labels)

In [8]:
train_data

array([[[[0.29411765, 0.30588235, 0.31372549],
         [0.29019608, 0.29803922, 0.30588235],
         [0.33333333, 0.3372549 , 0.32941176],
         ...,
         [0.26666667, 0.29411765, 0.29019608],
         [0.25490196, 0.27058824, 0.26666667],
         [0.25882353, 0.2627451 , 0.25882353]],

        [[0.3254902 , 0.32941176, 0.3372549 ],
         [0.31372549, 0.31372549, 0.32156863],
         [0.34509804, 0.34509804, 0.3254902 ],
         ...,
         [0.28627451, 0.30196078, 0.30588235],
         [0.29803922, 0.30588235, 0.29411765],
         [0.31372549, 0.31372549, 0.30588235]],

        [[0.30588235, 0.30588235, 0.31372549],
         [0.3372549 , 0.33333333, 0.3372549 ],
         [0.35294118, 0.34901961, 0.35294118],
         ...,
         [0.27843137, 0.29019608, 0.27843137],
         [0.28627451, 0.29019608, 0.27058824],
         [0.30588235, 0.30588235, 0.29019608]],

        ...,

        [[0.52156863, 0.5254902 , 0.54509804],
         [0.48235294, 0.49803922, 0.51764706]

In [9]:
labels

array([ 0,  0,  0, ..., 42, 42, 42])

In [11]:
train_data.shape, labels.shape

((39209, 30, 30, 3), (39209,))

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(train_data,labels,test_size=0.2,random_state=42)

In [17]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((31367, 30, 30, 3), (7842, 30, 30, 3), (31367,), (7842,))

In [25]:
Y_train = to_categorical(Y_train, 43)
Y_test = to_categorical(Y_test, 43)

In [26]:
#model building

model = Sequential([
    Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=X_train.shape[1:]),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(filters=128,kernel_size=(3,3),activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(256,activation='relu'),
    Dropout(0.5),
    Dense(43,activation='softmax'),
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [27]:
model.fit(X_train,Y_train,batch_size=64,epochs=15,validation_data=(X_test,Y_test))

Epoch 1/15
491/491 [==============================] - 15s 11ms/step - loss: 1.6683 - accuracy: 0.5268 - val_loss: 0.4043 - val_accuracy: 0.8902
Epoch 2/15
491/491 [==============================] - 5s 10ms/step - loss: 0.4655 - accuracy: 0.8567 - val_loss: 0.1366 - val_accuracy: 0.9717
Epoch 3/15
491/491 [==============================] - 5s 10ms/step - loss: 0.2822 - accuracy: 0.9137 - val_loss: 0.0776 - val_accuracy: 0.9852
Epoch 4/15
491/491 [==============================] - 5s 10ms/step - loss: 0.1908 - accuracy: 0.9427 - val_loss: 0.0585 - val_accuracy: 0.9888
Epoch 5/15
491/491 [==============================] - 5s 10ms/step - loss: 0.1557 - accuracy: 0.9524 - val_loss: 0.0410 - val_accuracy: 0.9922
Epoch 6/15
491/491 [==============================] - 6s 12ms/step - loss: 0.1290 - accuracy: 0.9610 - val_loss: 0.0436 - val_accuracy: 0.9906
Epoch 7/15
491/491 [==============================] - 15s 29ms/step - loss: 0.1069 - accuracy: 0.9672 - val_loss: 0.0318 - val_accuracy: 0.99

In [28]:
model.save("traffic_sign.h5")

In [32]:
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('./dataset/Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open("./dataset/"+img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
#Accuracy with the test data
print(accuracy_score(labels, pred))

0.9409342834520982
